In [45]:
from pyspark import SparkContext, SparkConf
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *

from pyspark.sql.types import (
                                StructType,
                                StructField,
                                IntegerType,
                                StringType,
                                FloatType,
                                Row
                            )

In [46]:
conf = SparkConf().set("spark.ui.port", "4050")

sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [47]:
#spark = SparkContext(master='local', appName='DataFrames')
sqlContext = SQLContext(spark)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


### Crear esquema

In [48]:
! head ./data/juegos.csv

,nombre_juego,annio,temporada,ciudad
1,1896 Verano,1896,Verano,Athina
2,1900 Verano,1900,Verano,Paris
3,1904 Verano,1904,Verano,St. Louis
4,1906 Verano,1906,Verano,Athina
5,1908 Verano,1908,Verano,London
6,1912 Verano,1912,Verano,Stockholm
7,1920 Verano,1920,Verano,Antwerpen
8,1924 Invierno,1924,Invierno,Chamonix
9,1924 Verano,1924,Verano,Paris


In [49]:
juegoSchema = StructType(
    [
        StructField(
            "juego_id", IntegerType(), True
        ),
        
        StructField(
            "anio", StringType(), False
        ),
        
        StructField(
            "temporada", StringType(), False
        ),
        
        StructField(
            "ciudad", StringType(), False
        )
    ]
)

### Crear dataframe desde un archivo `.csv` y usando un esquema

In [50]:
juegosDF = sqlContext.read.schema(juegoSchema)\
    .option("header", "true").csv("./data/juegos.csv")

juegosDF.show()

22/09/15 15:41:14 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 5, schema size: 4
CSV file: file:///workspace/data/juegos.csv
+--------+-------------+---------+--------+
|juego_id|         anio|temporada|  ciudad|
+--------+-------------+---------+--------+
|       1|  1896 Verano|     1896|  Verano|
|       2|  1900 Verano|     1900|  Verano|
|       3|  1904 Verano|     1904|  Verano|
|       4|  1906 Verano|     1906|  Verano|
|       5|  1908 Verano|     1908|  Verano|
|       6|  1912 Verano|     1912|  Verano|
|       7|  1920 Verano|     1920|  Verano|
|       8|1924 Invierno|     1924|Invierno|
|       9|  1924 Verano|     1924|  Verano|
|      10|1928 Invierno|     1928|Invierno|
|      11|  1928 Verano|     1928|  Verano|
|      12|1932 Invierno|     1932|Invierno|
|      13|  1932 Verano|     1932|  Verano|
|      14|1936 Invierno|     1936|Invierno|
|      15|  1936 Verano|     1936|  Verano|
|      16|

### Inferencia tipo de datos

In [51]:
player1 = sc.textFile('./data/player.csv').map(lambda x: x.split(','))
player2 = sc.textFile('./data/player2.csv').map(lambda x: x.split(','))

playerRDD = player1.union(player2)
playerRDD.take(3)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199']]

#### Quitar el encabezado

In [52]:
def remove_header(index, interator):
    return iter(list(interator)[1:])

In [53]:
playerRDDwoHead = playerRDD.mapPartitionsWithIndex(remove_header)
playerRDDwoHead.take(3)

[['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273']]

### Transformar datos

In [54]:
playerRDDwoHeadTrans = playerRDDwoHead.map(
    lambda x: (
        int(x[0]),
        x[1],
        int(x[2]),
        int(x[3]),
        int(x[4]),
        float(x[5]),
        int(x[6]),
    )
)
playerRDDwoHeadTrans.take(3)

[(1, 'A Dijiang', 1, 24, 180, 80.0, 199),
 (2, 'A Lamusi', 1, 23, 170, 60.0, 199),
 (3, 'Gunnar Nielsen Aaby', 1, 24, 0, 0.0, 273)]

### Crear un dataframe desde un RDD

In [55]:
player_schema = StructType(
    [
        StructField('deportista_id', IntegerType(), False),
        StructField('nombre', StringType(), False),
        StructField('genero', IntegerType(), False),
        StructField('edad', IntegerType(), False),
        StructField('altura', IntegerType(), False),
        StructField('peso', FloatType(), False),
        StructField('equipo_id', IntegerType(), False),
        
    ]
)

In [56]:
playerDF = sqlContext.createDataFrame(playerRDDwoHeadTrans, player_schema)
playerDF.show()

+-------------+--------------------+------+----+------+-----+---------+
|deportista_id|              nombre|genero|edad|altura| peso|equipo_id|
+-------------+--------------------+------+----+------+-----+---------+
|            1|           A Dijiang|     1|  24|   180| 80.0|      199|
|            2|            A Lamusi|     1|  23|   170| 60.0|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|     0|  0.0|      273|
|            4|Edgar Lindenau Aabye|     1|  34|     0|  0.0|      278|
|            5|Christine Jacoba ...|     2|  21|   185| 82.0|      705|
|            6|     Per Knut Aaland|     1|  31|   188| 75.0|     1096|
|            7|        John Aalberg|     1|  31|   183| 72.0|     1096|
|            8|Cornelia Cor Aalt...|     2|  18|   168|  0.0|      705|
|            9|    Antti Sami Aalto|     1|  26|   186| 96.0|      350|
|           10|Einar Ferdinand E...|     1|  26|     0|  0.0|      350|
|           11|  Jorma Ilmari Aalto|     1|  22|   182| 76.5|   

#### Cargar el RDD para países

In [57]:
countriesRDD = sc.textFile('./data/paises.csv').map(lambda x: x.split(','))
countriesRDDwoHead = countriesRDD.mapPartitionsWithIndex(remove_header)

countriesRDDwoHeadTrans = countriesRDDwoHead.map(
    lambda x:(
        int(x[0]),
        x[1],
        x[2]
    )
)

country_schema = StructType(
    [
        StructField('id', IntegerType(), False),
        StructField('equipo', StringType(), False),
        StructField('sigla', StringType(), False)
    ]
)
countriesDF = sqlContext.createDataFrame(countriesRDDwoHeadTrans, country_schema)
countriesDF.show()

+---+--------------------+-----+
| id|              equipo|sigla|
+---+--------------------+-----+
|  1|         30. Februar|  AUT|
|  2|A North American ...|  MEX|
|  3|           Acipactli|  MEX|
|  4|             Acturus|  ARG|
|  5|         Afghanistan|  AFG|
|  6|            Akatonbo|  IRL|
|  7|            Alain IV|  SUI|
|  8|             Albania|  ALB|
|  9|              Alcaid|  POR|
| 10|            Alcyon-6|  FRA|
| 11|            Alcyon-7|  FRA|
| 12|           Aldebaran|  ITA|
| 13|        Aldebaran II|  ITA|
| 14|              Aletta|  IRL|
| 15|             Algeria|  ALG|
| 16|         Ali-Baba II|  SWE|
| 17|         Ali-Baba IV|  SUI|
| 18|         Ali-Baba IX|  SUI|
| 19|         Ali-Baba VI|  SUI|
| 20|             Allegro|  FRA|
+---+--------------------+-----+
only showing top 20 rows



In [58]:
! head ./data/deporte.csv

deporte_id,deporte
1,Basketball
2,Judo
3,Football
4,Tug-Of-War
5,Speed Skating
6,Cross Country Skiing
7,Athletics
8,Ice Hockey
9,Swimming


In [59]:
sportRDD = sc.textFile('./data/deporte.csv').map(lambda x: x.split(','))
sportRDDwoHead = sportRDD.mapPartitionsWithIndex(remove_header)

sportRDDwoHeadTrans = sportRDDwoHead.map(
    lambda x: (
        int(x[0]),
        str(x[1])
    )
)

sport_schema = StructType(
    [
        StructField('deporte_id', IntegerType(), False),
        StructField('deporte', StringType(), False)
    ]
)

sportDF = sqlContext.createDataFrame(sportRDDwoHeadTrans, sport_schema)
sportDF.show()

+----------+--------------------+
|deporte_id|             deporte|
+----------+--------------------+
|         1|          Basketball|
|         2|                Judo|
|         3|            Football|
|         4|          Tug-Of-War|
|         5|       Speed Skating|
|         6|Cross Country Skiing|
|         7|           Athletics|
|         8|          Ice Hockey|
|         9|            Swimming|
|        10|           Badminton|
|        11|             Sailing|
|        12|            Biathlon|
|        13|          Gymnastics|
|        14|    Art Competitions|
|        15|       Alpine Skiing|
|        16|            Handball|
|        17|       Weightlifting|
|        18|           Wrestling|
|        19|                Luge|
|        20|          Water Polo|
+----------+--------------------+
only showing top 20 rows



In [60]:
! head ./data/evento.csv

evento_id,evento,deporte_id
1,Basketball Men's Basketball,1
2,Judo Men's Extra-Lightweight,2
3,Football Men's Football,3
4,Tug-Of-War Men's Tug-Of-War,4
5,Speed Skating Women's 500 metres,5
6,"Speed Skating Women's 1,000 metres",5
7,Cross Country Skiing Men's 10 kilometres,6
8,Cross Country Skiing Men's 50 kilometres,6
9,Cross Country Skiing Men's 10/15 kilometres Pursuit,6


#### Forma 1 con transformación del RDD

#### Forma 2 leyendo directamente el archivo `csv`

In [66]:
event_schema = StructType(
    [
        StructField('evento_id', IntegerType(), False),
        StructField('nombre', StringType(), False),
        StructField('deporte_id', IntegerType(), False)
    ]
)
eventDF = sqlContext.read.schema(event_schema).option('header', True).csv('./data/evento.csv')
eventDF.show()

22/09/15 15:42:17 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: evento_id, evento, deporte_id
 Schema: evento_id, nombre, deporte_id
Expected: nombre but found: evento
CSV file: file:///workspace/data/evento.csv
+---------+--------------------+----------+
|evento_id|              nombre|deporte_id|
+---------+--------------------+----------+
|        1|Basketball Men's ...|         1|
|        2|Judo Men's Extra-...|         2|
|        3|Football Men's Fo...|         3|
|        4|Tug-Of-War Men's ...|         4|
|        5|Speed Skating Wom...|         5|
|        6|Speed Skating Wom...|         5|
|        7|Cross Country Ski...|         6|
|        8|Cross Country Ski...|         6|
|        9|Cross Country Ski...|         6|
|       10|Cross Country Ski...|         6|
|       11|Cross Country Ski...|         6|
|       12|Athletics Women's...|         7|
|       13|Athletics Women's...|         7|
|       14|Ice Hockey Men's ...|         8|
|       15|

In [62]:
! head ./data/resultados.csv

resultado_id,medalla,deportista_id,juego_id,evento_id
1,NA,1,39,1
2,NA,2,49,2
3,NA,3,7,3
4,Gold,4,2,4
5,NA,5,36,5
6,NA,5,36,6
7,NA,5,38,5
8,NA,5,38,6
9,NA,5,40,5


In [63]:
result_schema = StructType(
    [
        StructField('resultado_id', IntegerType(), False),
        StructField('medalla', StringType(), False),
        StructField('deportista_id', IntegerType(), False),
        StructField('juego_id', IntegerType(), False),
        StructField('evento_id', IntegerType(), False),
    ]
)

resultDF = sqlContext.read.schema(result_schema).option('header', True).csv('./data/resultados.csv')
resultDF.show()

+------------+-------+-------------+--------+---------+
|resultado_id|medalla|deportista_id|juego_id|evento_id|
+------------+-------+-------------+--------+---------+
|           1|     NA|            1|      39|        1|
|           2|     NA|            2|      49|        2|
|           3|     NA|            3|       7|        3|
|           4|   Gold|            4|       2|        4|
|           5|     NA|            5|      36|        5|
|           6|     NA|            5|      36|        6|
|           7|     NA|            5|      38|        5|
|           8|     NA|            5|      38|        6|
|           9|     NA|            5|      40|        5|
|          10|     NA|            5|      40|        6|
|          11|     NA|            6|      38|        7|
|          12|     NA|            6|      38|        8|
|          13|     NA|            6|      38|        9|
|          14|     NA|            6|      38|       10|
|          15|     NA|            6|      40|   

In [64]:
sportDF.take(3)

[Row(deporte_id=1, deporte='Basketball'),
 Row(deporte_id=2, deporte='Judo'),
 Row(deporte_id=3, deporte='Football')]

In [68]:
eventDF.take(3)

22/09/15 15:42:40 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: evento_id, evento, deporte_id
 Schema: evento_id, nombre, deporte_id
Expected: nombre but found: evento
CSV file: file:///workspace/data/evento.csv


[Row(evento_id=1, nombre="Basketball Men's Basketball", deporte_id=1),
 Row(evento_id=2, nombre="Judo Men's Extra-Lightweight", deporte_id=2),
 Row(evento_id=3, nombre="Football Men's Football", deporte_id=3)]

In [65]:
countriesDF.take(3)

[Row(id=1, equipo='30. Februar', sigla='AUT'),
 Row(id=2, equipo='A North American Team', sigla='MEX'),
 Row(id=3, equipo='Acipactli', sigla='MEX')]

In [69]:
juegosDF.take(3)

22/09/15 15:44:46 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 5, schema size: 4
CSV file: file:///workspace/data/juegos.csv


[Row(juego_id=1, anio='1896 Verano', temporada='1896', ciudad='Verano'),
 Row(juego_id=2, anio='1900 Verano', temporada='1900', ciudad='Verano'),
 Row(juego_id=3, anio='1904 Verano', temporada='1904', ciudad='Verano')]

In [70]:
playerDF.take(3)

[Row(deportista_id=1, nombre='A Dijiang', genero=1, edad=24, altura=180, peso=80.0, equipo_id=199),
 Row(deportista_id=2, nombre='A Lamusi', genero=1, edad=23, altura=170, peso=60.0, equipo_id=199),
 Row(deportista_id=3, nombre='Gunnar Nielsen Aaby', genero=1, edad=24, altura=0, peso=0.0, equipo_id=273)]

In [71]:
resultDF.take(3)

[Row(resultado_id=1, medalla='NA', deportista_id=1, juego_id=39, evento_id=1),
 Row(resultado_id=2, medalla='NA', deportista_id=2, juego_id=49, evento_id=2),
 Row(resultado_id=3, medalla='NA', deportista_id=3, juego_id=7, evento_id=3)]

In [72]:
playerDF.printSchema()

root
 |-- deportista_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- genero: integer (nullable = false)
 |-- edad: integer (nullable = false)
 |-- altura: integer (nullable = false)
 |-- peso: float (nullable = false)
 |-- equipo_id: integer (nullable = false)



### Renombrar y eliminar columnas en un DF

#### Way 1

In [73]:
playerDF.withColumnRenamed("genero", "sexo").drop('altura').printSchema()

root
 |-- deportista_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- sexo: integer (nullable = false)
 |-- edad: integer (nullable = false)
 |-- peso: float (nullable = false)
 |-- equipo_id: integer (nullable = false)



#### Way 2

In [74]:
from pyspark.sql.functions import col

playerDFf = playerDF.select(
    "deportista_id",
    "nombre",
    col("edad").alias("edad_al_jugar"),
    "equipo_id"
    )
playerDFf.show()

+-------------+--------------------+-------------+---------+
|deportista_id|              nombre|edad_al_jugar|equipo_id|
+-------------+--------------------+-------------+---------+
|            1|           A Dijiang|           24|      199|
|            2|            A Lamusi|           23|      199|
|            3| Gunnar Nielsen Aaby|           24|      273|
|            4|Edgar Lindenau Aabye|           34|      278|
|            5|Christine Jacoba ...|           21|      705|
|            6|     Per Knut Aaland|           31|     1096|
|            7|        John Aalberg|           31|     1096|
|            8|Cornelia Cor Aalt...|           18|      705|
|            9|    Antti Sami Aalto|           26|      350|
|           10|Einar Ferdinand E...|           26|      350|
|           11|  Jorma Ilmari Aalto|           22|      350|
|           12|   Jyri Tapani Aalto|           31|      350|
|           13|  Minna Maarit Aalto|           30|      350|
|           14|Pirjo Han

In [76]:
playerDFf.sort('edad_al_jugar').show()

+-------------+--------------------+-------------+---------+
|deportista_id|              nombre|edad_al_jugar|equipo_id|
+-------------+--------------------+-------------+---------+
|          224|     Mohamed AbdelEl|            0|      308|
|          487|      Inni Aboubacar|            0|      721|
|          226|Sanad Bushara Abd...|            0|     1003|
|           58|    Georgi Abadzhiev|            0|      154|
|          230|    Moustafa Abdelal|            0|      308|
|          102|   Sayed Fahmy Abaza|            0|      308|
|          260|  Ahmed Abdo Mustafa|            0|     1003|
|          139|George Ioannis Abbot|            0|     1043|
|          281|      S. Abdul Hamid|            0|      487|
|          163|     Ismail Abdallah|            0|     1095|
|          285|Talal Hassoun Abd...|            0|      497|
|          173| Mohamed Abdel Fatah|            0|     1003|
|          179|Ibrahim Saad Abde...|            0|     1003|
|          378|     Ange

### Aplicar filtros al DF

In [78]:
playerDFf.filter(
    (playerDFf.edad_al_jugar !=0)
).show()

+-------------+--------------------+-------------+---------+
|deportista_id|              nombre|edad_al_jugar|equipo_id|
+-------------+--------------------+-------------+---------+
|            1|           A Dijiang|           24|      199|
|            2|            A Lamusi|           23|      199|
|            3| Gunnar Nielsen Aaby|           24|      273|
|            4|Edgar Lindenau Aabye|           34|      278|
|            5|Christine Jacoba ...|           21|      705|
|            6|     Per Knut Aaland|           31|     1096|
|            7|        John Aalberg|           31|     1096|
|            8|Cornelia Cor Aalt...|           18|      705|
|            9|    Antti Sami Aalto|           26|      350|
|           10|Einar Ferdinand E...|           26|      350|
|           11|  Jorma Ilmari Aalto|           22|      350|
|           12|   Jyri Tapani Aalto|           31|      350|
|           13|  Minna Maarit Aalto|           30|      350|
|           14|Pirjo Han

In [79]:
playerDFf.filter(
    (playerDFf.edad_al_jugar !=0)
).sort('edad_al_jugar').show()

+-------------+--------------------+-------------+---------+
|deportista_id|              nombre|edad_al_jugar|equipo_id|
+-------------+--------------------+-------------+---------+
|        71691|  Dimitrios Loundras|           10|      333|
|        40129|    Luigina Giavotti|           11|      507|
|        47618|Sonja Henie Toppi...|           11|      742|
|        51268|      Beatrice Hutiu|           11|      861|
|       118925|Megan Olwen Deven...|           11|      413|
|        22411|Magdalena Cecilia...|           11|      413|
|       126307|        Liana Vicens|           11|      825|
|        70616|          Liu Luyang|           11|      199|
|        37333|Carlos Bienvenido...|           11|      982|
|        76675|   Marcelle Matthews|           11|      967|
|        52070|        Etsuko Inada|           11|      514|
|        74755|Luciana Marcellin...|           12|      507|
|       118832|  Chin Say Molly Tay|           12|      619|
|        72854|      Lic

In [80]:
playerDF.printSchema()

root
 |-- deportista_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- genero: integer (nullable = false)
 |-- edad: integer (nullable = false)
 |-- altura: integer (nullable = false)
 |-- peso: float (nullable = false)
 |-- equipo_id: integer (nullable = false)



In [81]:
resultDF.printSchema()

root
 |-- resultado_id: integer (nullable = true)
 |-- medalla: string (nullable = true)
 |-- deportista_id: integer (nullable = true)
 |-- juego_id: integer (nullable = true)
 |-- evento_id: integer (nullable = true)



In [82]:
juegosDF.printSchema()

root
 |-- juego_id: integer (nullable = true)
 |-- anio: string (nullable = true)
 |-- temporada: string (nullable = true)
 |-- ciudad: string (nullable = true)



### Agrupaciones y join sobre DF

In [83]:
playerDF.join(
    resultDF,
    playerDF.deportista_id == resultDF.deportista_id,
    "left"
).show(5)

+-------------+--------------------+------+----+------+----+---------+------------+-------+-------------+--------+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|resultado_id|medalla|deportista_id|juego_id|evento_id|
+-------------+--------------------+------+----+------+----+---------+------------+-------+-------------+--------+---------+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|           1|     NA|            1|      39|        1|
|            2|            A Lamusi|     1|  23|   170|60.0|      199|           2|     NA|            2|      49|        2|
|            3| Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|           3|     NA|            3|       7|        3|
|            4|Edgar Lindenau Aabye|     1|  34|     0| 0.0|      278|           4|   Gold|            4|       2|        4|
|            5|Christine Jacoba ...|     2|  21|   185|82.0|      705|          10|     NA|            5|      40|        6|


In [84]:
playerDF.join(
    resultDF,
    playerDF.deportista_id == resultDF.deportista_id,
    "left"
    ).join(
        juegosDF,
        juegosDF.juego_id == resultDF.juego_id,
        "left"
        ).show(5)

22/09/15 16:34:53 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 5, schema size: 4
CSV file: file:///workspace/data/juegos.csv


+-------------+--------------------+------+----+------+----+---------+------------+-------+-------------+--------+---------+--------+-------------+---------+--------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|resultado_id|medalla|deportista_id|juego_id|evento_id|juego_id|         anio|temporada|  ciudad|
+-------------+--------------------+------+----+------+----+---------+------------+-------+-------------+--------+---------+--------+-------------+---------+--------+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|           1|     NA|            1|      39|        1|      39|  1992 Verano|     1992|  Verano|
|            2|            A Lamusi|     1|  23|   170|60.0|      199|           2|     NA|            2|      49|        2|      49|  2012 Verano|     2012|  Verano|
|            3| Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|           3|     NA|            3|       7|        3|       7|  1920 Verano|     1920|  Verano

In [87]:
playerDF.join(
    resultDF,
    playerDF.deportista_id == resultDF.deportista_id,
    "left"
    ).join(
        juegosDF,
        juegosDF.juego_id == resultDF.juego_id,
        "left"
        ).join(
            eventDF,
            eventDF.evento_id == resultDF.evento_id,
            "left"
            ).show(5)

22/09/15 16:37:24 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 5, schema size: 4
CSV file: file:///workspace/data/juegos.csv
22/09/15 16:37:24 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: evento_id, evento, deporte_id
 Schema: evento_id, nombre, deporte_id
Expected: nombre but found: evento
CSV file: file:///workspace/data/evento.csv
+-------------+--------------------+------+----+------+----+---------+------------+-------+-------------+--------+---------+--------+-------------+---------+--------+---------+--------------------+----------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|resultado_id|medalla|deportista_id|juego_id|evento_id|juego_id|         anio|temporada|  ciudad|evento_id|              nombre|deporte_id|
+-------------+--------------------+------+----+------+----+---------+------------+-------+-------------+--------+---------+--------+----------

In [89]:
playerDF.join(
    resultDF,
    playerDF.deportista_id == resultDF.deportista_id,
    "left"
    ).join(
        juegosDF,
        juegosDF.juego_id == resultDF.juego_id,
        "left"
        ).join(
            eventDF,
            eventDF.evento_id == resultDF.evento_id,
            "left"
            ).select(
                playerDF.nombre,
                col("edad").alias("edad_al_jugar"),
                "medalla",
                col("anio").alias('year_del_juego'),
                eventDF.nombre.alias('nombre_disciplina')
                ).show(5)

22/09/15 16:40:54 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , nombre_juego
 Schema: juego_id, anio
Expected: juego_id but found: 
CSV file: file:///workspace/data/juegos.csv
22/09/15 16:40:54 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: evento_id, evento
 Schema: evento_id, nombre
Expected: nombre but found: evento
CSV file: file:///workspace/data/evento.csv
+--------------------+-------------+-------+--------------+--------------------+
|              nombre|edad_al_jugar|medalla|year_del_juego|   nombre_disciplina|
+--------------------+-------------+-------+--------------+--------------------+
|           A Dijiang|           24|     NA|   1992 Verano|Basketball Men's ...|
|            A Lamusi|           23|     NA|   2012 Verano|Judo Men's Extra-...|
| Gunnar Nielsen Aaby|           24|     NA|   1920 Verano|Football Men's Fo...|
|Edgar Lindenau Aabye|           34|   Gold|   1900 Verano|Tug-Of-War Men's ...|
|Christin

### Reto

In [92]:
resultDF.filter(resultDF.medalla != 'NA').join(
    playerDF,
    playerDF.deportista_id == resultDF.deportista_id,
    "left"
    ).show(5)

+------------+-------+-------------+--------+---------+-------------+--------------------+------+----+------+----+---------+
|resultado_id|medalla|deportista_id|juego_id|evento_id|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+------------+-------+-------------+--------+---------+-------------+--------------------+------+----+------+----+---------+
|          41| Bronze|           16|      50|       14|           16|Juhamatti Tapio A...|     1|  28|   184|85.0|      350|
|          38| Bronze|           15|       7|       19|           15|Arvo Ossian Aaltonen|     1|  22|     0| 0.0|      350|
|          39| Bronze|           15|       7|       20|           15|Arvo Ossian Aaltonen|     1|  22|     0| 0.0|      350|
|          42| Bronze|           17|      17|       21|           17|Paavo Johannes Aa...|     1|  28|   175|64.0|      350|
|          43|   Gold|           17|      17|       22|           17|Paavo Johannes Aa...|     1|  28|   175|64.0|      350|


In [95]:
resultDF.filter(resultDF.medalla != 'NA').join(
    playerDF,
    playerDF.deportista_id == resultDF.deportista_id,
    "left"
    ).join(
        countriesDF,
        countriesDF.id == playerDF.equipo_id
        ).show(5)

+------------+-------+-------------+--------+---------+-------------+--------------------+------+----+------+----+---------+---+--------------+-----+
|resultado_id|medalla|deportista_id|juego_id|evento_id|deportista_id|              nombre|genero|edad|altura|peso|equipo_id| id|        equipo|sigla|
+------------+-------+-------------+--------+---------+-------------+--------------------+------+----+------+----+---------+---+--------------+-----+
|           4|   Gold|            4|       2|        4|            4|Edgar Lindenau Aabye|     1|  34|     0| 0.0|      278|278|Denmark/Sweden|  SWE|
|          39| Bronze|           15|       7|       20|           15|Arvo Ossian Aaltonen|     1|  22|     0| 0.0|      350|350|       Finland|  FIN|
|          38| Bronze|           15|       7|       19|           15|Arvo Ossian Aaltonen|     1|  22|     0| 0.0|      350|350|       Finland|  FIN|
|          41| Bronze|           16|      50|       14|           16|Juhamatti Tapio A...|     1|  2

In [100]:
resultDF.filter(resultDF.medalla != 'NA').join(
    playerDF,
    playerDF.deportista_id == resultDF.deportista_id,
    "left"
    ).join(
        countriesDF,
        countriesDF.id == playerDF.equipo_id
        ).select(
            "medalla",
            "equipo",
            "sigla"
            ).sort(
                col("sigla").desc()
                ).show()

+-------+--------+-----+
|medalla|  equipo|sigla|
+-------+--------+-----+
|   Gold|Zimbabwe|  ZIM|
| Bronze|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
+-------+--------+-----+
only showing top 20 rows



### Funciones de agrupación

In [102]:
playerDF.join(
    resultDF,
    resultDF.deportista_id == playerDF.deportista_id,
    "left"
    ).show(5)

+-------------+--------------------+------+----+------+----+---------+------------+-------+-------------+--------+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|resultado_id|medalla|deportista_id|juego_id|evento_id|
+-------------+--------------------+------+----+------+----+---------+------------+-------+-------------+--------+---------+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|           1|     NA|            1|      39|        1|
|            2|            A Lamusi|     1|  23|   170|60.0|      199|           2|     NA|            2|      49|        2|
|            3| Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|           3|     NA|            3|       7|        3|
|            4|Edgar Lindenau Aabye|     1|  34|     0| 0.0|      278|           4|   Gold|            4|       2|        4|
|            5|Christine Jacoba ...|     2|  21|   185|82.0|      705|          10|     NA|            5|      40|        6|


In [104]:
playerDF.join(
    resultDF,
    resultDF.deportista_id == playerDF.deportista_id,
    "left"
    ).join(
        juegosDF,
        juegosDF.juego_id == resultDF.juego_id,
        "left"
        ).show(5)

22/09/15 16:54:50 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 5, schema size: 4
CSV file: file:///workspace/data/juegos.csv


+-------------+--------------------+------+----+------+----+---------+------------+-------+-------------+--------+---------+--------+-------------+---------+--------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|resultado_id|medalla|deportista_id|juego_id|evento_id|juego_id|         anio|temporada|  ciudad|
+-------------+--------------------+------+----+------+----+---------+------------+-------+-------------+--------+---------+--------+-------------+---------+--------+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|           1|     NA|            1|      39|        1|      39|  1992 Verano|     1992|  Verano|
|            2|            A Lamusi|     1|  23|   170|60.0|      199|           2|     NA|            2|      49|        2|      49|  2012 Verano|     2012|  Verano|
|            3| Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|           3|     NA|            3|       7|        3|       7|  1920 Verano|     1920|  Verano

In [105]:
playerDF.join(
    resultDF,
    resultDF.deportista_id == playerDF.deportista_id,
    "left"
    ).join(
        juegosDF,
        juegosDF.juego_id == resultDF.juego_id,
        "left"
        ).join(
            countriesDF,
            playerDF.equipo_id == countriesDF.id,
            "left"
            ).show(5)

22/09/15 16:56:28 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 5, schema size: 4
CSV file: file:///workspace/data/juegos.csv
+-------------+--------------------+------+----+------+----+---------+------------+-------+-------------+--------+---------+--------+-------------+---------+--------+---+--------------+-----+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|resultado_id|medalla|deportista_id|juego_id|evento_id|juego_id|         anio|temporada|  ciudad| id|        equipo|sigla|
+-------------+--------------------+------+----+------+----+---------+------------+-------+-------------+--------+---------+--------+-------------+---------+--------+---+--------------+-----+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|           1|     NA|            1|      39|        1|      39|  1992 Verano|     1992|  Verano|199|         China|  CHN|
|            2|            A Lamus

In [107]:
playerDF.join(
    resultDF,
    resultDF.deportista_id == playerDF.deportista_id,
    "left"
    ).join(
        juegosDF,
        juegosDF.juego_id == resultDF.juego_id,
        "left"
        ).join(
            countriesDF,
            playerDF.equipo_id == countriesDF.id,
            "left"
            ).join(
                eventDF,
                eventDF.evento_id == resultDF.evento_id,
                "left"
                ).show(5)

22/09/15 16:57:53 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: evento_id, evento, deporte_id
 Schema: evento_id, nombre, deporte_id
Expected: nombre but found: evento
CSV file: file:///workspace/data/evento.csv
22/09/15 16:57:53 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 5, schema size: 4
CSV file: file:///workspace/data/juegos.csv
+-------------+--------------------+------+----+------+----+---------+------------+-------+-------------+--------+---------+--------+-------------+---------+--------+---+--------------+-----+---------+--------------------+----------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|resultado_id|medalla|deportista_id|juego_id|evento_id|juego_id|         anio|temporada|  ciudad| id|        equipo|sigla|evento_id|              nombre|deporte_id|
+-------------+--------------------+------+----+------+----+---------+------------+-------+--

In [109]:
playerDF.join(
    resultDF,
    resultDF.deportista_id == playerDF.deportista_id,
    "left"
    ).join(
        juegosDF,
        juegosDF.juego_id == resultDF.juego_id,
        "left"
        ).join(
            countriesDF,
            playerDF.equipo_id == countriesDF.id,
            "left"
            ).join(
                eventDF,
                eventDF.evento_id == resultDF.evento_id,
                "left"
                ).join(
                    sportDF,
                    sportDF.deporte_id == eventDF.deporte_id,
                    "left"
                    ).show(5)

22/09/15 16:58:56 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: evento_id, evento, deporte_id
 Schema: evento_id, nombre, deporte_id
Expected: nombre but found: evento
CSV file: file:///workspace/data/evento.csv
22/09/15 16:58:56 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 5, schema size: 4
CSV file: file:///workspace/data/juegos.csv
+-------------+--------------------+------+----+------+----+---------+------------+-------+-------------+--------+---------+--------+-------------+---------+--------+---+--------------+-----+---------+--------------------+----------+----------+-------------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|resultado_id|medalla|deportista_id|juego_id|evento_id|juego_id|         anio|temporada|  ciudad| id|        equipo|sigla|evento_id|              nombre|deporte_id|deporte_id|      deporte|
+-------------+--------------------+------+

In [114]:
playerDF.join(
    resultDF,
    resultDF.deportista_id == playerDF.deportista_id,
    "left"
    ).join(
        juegosDF,
        juegosDF.juego_id == resultDF.juego_id,
        "left"
        ).join(
            countriesDF,
            playerDF.equipo_id == countriesDF.id,
            "left"
            ).join(
                eventDF,
                eventDF.evento_id == resultDF.evento_id,
                "left"
                ).join(
                    sportDF,
                    sportDF.deporte_id == eventDF.deporte_id,
                    "left"
                    ).select(
                        "sigla",
                        col("anio").alias("year"),
                        "medalla",
                        eventDF.nombre.alias("nombre_subdisciplina"),
                        sportDF.deporte.alias("nombre_disciplina"),
                        playerDF.nombre
                        ).show(5)

22/09/15 17:02:59 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: evento_id, evento, deporte_id
 Schema: evento_id, nombre, deporte_id
Expected: nombre but found: evento
CSV file: file:///workspace/data/evento.csv
22/09/15 17:03:00 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , nombre_juego
 Schema: juego_id, anio
Expected: juego_id but found: 
CSV file: file:///workspace/data/juegos.csv
+-----+-------------+-------+--------------------+-----------------+--------------------+
|sigla|         year|medalla|nombre_subdisciplina|nombre_disciplina|              nombre|
+-----+-------------+-------+--------------------+-----------------+--------------------+
|  CHN|  1992 Verano|     NA|Basketball Men's ...|       Basketball|           A Dijiang|
|  CHN|  2012 Verano|     NA|Judo Men's Extra-...|             Judo|            A Lamusi|
|  DEN|  1920 Verano|     NA|Football Men's Fo...|         Football| Gunnar Nielsen Aaby|
|  SWE|  190

In [110]:
eventDF.printSchema()

root
 |-- evento_id: integer (nullable = true)
 |-- nombre: string (nullable = true)
 |-- deporte_id: integer (nullable = true)



In [119]:
medals_per_year = playerDF.join(
    resultDF,
    playerDF.deportista_id == resultDF.deportista_id,
    "left"
    ).join(
        juegosDF,
        juegosDF.juego_id == resultDF.juego_id,
        "left"
        ).join(
            countriesDF,
            playerDF.equipo_id == countriesDF.id,
            "left"
            ).join(
                eventDF,
                eventDF.evento_id == resultDF.evento_id,
                "left"
                ).join(
                    sportDF,
                    eventDF.deporte_id == sportDF.deporte_id,
                    "left"
                    ).select(
                        "sigla",
                        col("anio").alias("year"),
                        "medalla",
                        eventDF.nombre.alias("nombre_subdisciplina"),
                        sportDF.deporte.alias("nombre_disciplina"),
                        playerDF.nombre
                        )
medals_per_year.show()

22/09/15 17:10:50 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , nombre_juego
 Schema: juego_id, anio
Expected: juego_id but found: 
CSV file: file:///workspace/data/juegos.csv
22/09/15 17:10:50 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: evento_id, evento, deporte_id
 Schema: evento_id, nombre, deporte_id
Expected: nombre but found: evento
CSV file: file:///workspace/data/evento.csv
+-----+-------------+-------+--------------------+--------------------+--------------------+
|sigla|         year|medalla|nombre_subdisciplina|   nombre_disciplina|              nombre|
+-----+-------------+-------+--------------------+--------------------+--------------------+
|  CHN|  1992 Verano|     NA|Basketball Men's ...|          Basketball|           A Dijiang|
|  CHN|  2012 Verano|     NA|Judo Men's Extra-...|                Judo|            A Lamusi|
|  DEN|  1920 Verano|     NA|Football Men's Fo...|            Football| Gunnar Nielsen 

In [117]:
medals_per_year.filter(medals_per_year.medalla != 'NA').show(5)

22/09/15 17:04:49 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , nombre_juego
 Schema: juego_id, anio
Expected: juego_id but found: 
CSV file: file:///workspace/data/juegos.csv
22/09/15 17:04:49 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: evento_id, evento, deporte_id
 Schema: evento_id, nombre, deporte_id
Expected: nombre but found: evento
CSV file: file:///workspace/data/evento.csv
+-----+-------------+-------+--------------------+-----------------+--------------------+
|sigla|         year|medalla|nombre_subdisciplina|nombre_disciplina|              nombre|
+-----+-------------+-------+--------------------+-----------------+--------------------+
|  SWE|  1900 Verano|   Gold|Tug-Of-War Men's ...|       Tug-Of-War|Edgar Lindenau Aabye|
|  FIN|  1920 Verano| Bronze|Swimming Men's 40...|         Swimming|Arvo Ossian Aaltonen|
|  FIN|  1920 Verano| Bronze|Swimming Men's 20...|         Swimming|Arvo Ossian Aaltonen|
|  FIN|2014 

### Agrupar por año

In [126]:
medals_x_year_gruped = medals_per_year.filter(
    medals_per_year.medalla != 'NA'
    ).groupBy(
        'sigla',
        'year',
        'nombre_subdisciplina'
        ).count().sort(
                    'year'
                    )

medals_x_year_gruped.show()

22/09/15 17:15:10 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , nombre_juego
 Schema: juego_id, anio
Expected: juego_id but found: 
CSV file: file:///workspace/data/juegos.csv
22/09/15 17:15:10 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: evento_id, evento, deporte_id
 Schema: evento_id, nombre, deporte_id
Expected: nombre but found: evento
CSV file: file:///workspace/data/evento.csv


+-----+-----------+--------------------+-----+
|sigla|       year|nombre_subdisciplina|count|
+-----+-----------+--------------------+-----+
|  GRE|1896 Verano|Swimming Men's 1,...|    1|
|  AUS|1896 Verano|Athletics Men's 8...|    1|
|  GRE|1896 Verano|Shooting Men's Mi...|    1|
|  USA|1896 Verano|Athletics Men's L...|    3|
|  GBR|1896 Verano|Tennis Men's Doubles|    2|
|  USA|1896 Verano|Athletics Men's 1...|    1|
|  GER|1896 Verano|Gymnastics Men's ...|   10|
|  GBR|1896 Verano|Weightlifting Men...|    1|
|  USA|1896 Verano|Athletics Men's H...|    3|
|  GBR|1896 Verano|Cycling Men's Roa...|    1|
|  FRA|1896 Verano|Fencing Men's Foi...|    2|
|  USA|1896 Verano|Athletics Men's 4...|    2|
|  USA|1896 Verano|Athletics Men's 1...|    2|
|  GRE|1896 Verano|Swimming Men's 10...|    3|
|  GRE|1896 Verano|Gymnastics Men's ...|    2|
|  GBR|1896 Verano|Tennis Men's Singles|    1|
|  GER|1896 Verano|Gymnastics Men's ...|   10|
|  USA|1896 Verano|Athletics Men's 1...|    1|
|  GBR|1896 V

In [127]:
medals_x_year_gruped.printSchema()

root
 |-- sigla: string (nullable = true)
 |-- year: string (nullable = true)
 |-- nombre_subdisciplina: string (nullable = true)
 |-- count: long (nullable = false)



In [128]:
medals_x_year_gruped.groupBy(
    "sigla",
    "year"
    ).agg(
        sum("count").alias("total_medals"),
        avg("count").alias('mean_medals')
        ).show()

22/09/15 17:17:32 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , nombre_juego
 Schema: juego_id, anio
Expected: juego_id but found: 
CSV file: file:///workspace/data/juegos.csv
22/09/15 17:17:32 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: evento_id, evento, deporte_id
 Schema: evento_id, nombre, deporte_id
Expected: nombre but found: evento
CSV file: file:///workspace/data/evento.csv


+-----+-------------+------------+------------------+
|sigla|         year|total_medals|       mean_medals|
+-----+-------------+------------+------------------+
|  MGL|  2008 Verano|           5|              1.25|
|  SUI|2014 Invierno|          29|3.2222222222222223|
|  ETH|  2004 Verano|           7|              1.75|
|  BEL|  2000 Verano|           7|               1.4|
|  SWE|  1976 Verano|          10|               2.0|
|  AUT|  1928 Verano|           5|              1.25|
|  SYR|  1984 Verano|           1|               1.0|
|  NED|1992 Invierno|           4|1.3333333333333333|
|  ITA|  1996 Verano|          69| 2.225806451612903|
|  ARG|  2004 Verano|          49| 8.166666666666666|
|  CHI|  1988 Verano|           1|               1.0|
|  URS|1984 Invierno|          56|               2.8|
|  DEN|  1956 Verano|           6|               1.5|
|  THA|  2008 Verano|           4|               1.0|
|  DEN|  1896 Verano|           6|               1.0|
|  GRN|  2016 Verano|       

### DF registrados como SQL

In [137]:
resultDF.createOrReplaceTempView('result')
playerDF.createOrReplaceTempView('player')
countriesDF.createOrReplaceTempView('countries')

In [138]:
sqlContext.sql("SELECT * FROM player").show(5)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|
|            2|            A Lamusi|     1|  23|   170|60.0|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|
|            4|Edgar Lindenau Aabye|     1|  34|     0| 0.0|      278|
|            5|Christine Jacoba ...|     2|  21|   185|82.0|      705|
+-------------+--------------------+------+----+------+----+---------+
only showing top 5 rows



In [139]:
sqlContext.sql(
    """
    SELECT
        medalla,
        equipo,
        sigla
    FROM
        result r    
    JOIN
        player p
        ON r.deportista_id = p.deportista_id
        
        JOIN
            countries c
            ON c.id = p.equipo_id
    WHERE
        medalla <> 'NA'
        
    ORDER BY
        sigla DESC
    """
).show()

+-------+--------+-----+
|medalla|  equipo|sigla|
+-------+--------+-----+
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Bronze|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
+-------+--------+-----+
only showing top 20 rows

